# Bootes master catalogue

This notebook presents the combination of the HELP standard catalogue with the aperture matched photometry from Brown which gives superior photometric redhsifts.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
0246c5d (Thu Jan 25 17:01:47 2018 +0000) [with local modifications]
This notebook was executed on: 
2018-05-01 12:55:44.427637


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

In [3]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [4]:

brown_catalogue = Table.read("{}/master_catalogue_bootes_brown_20180509.fits".format(OUT_DIR))
help_catalogue = Table.read("{}/master_catalogue_bootes_help_20180501.fits".format(OUT_DIR))


cross_id = Table.read("{}/master_list_cross_ident_bootes_brown_20180509.fits".format(OUT_DIR))['full_help_id', 'help_id']
cross_id['help_id'].name = 'brown_help_id'
cross_id['full_help_id'].name = 'help_help_id'



In [ ]:
in_brown = set(brown_catalogue.colnames)
in_help = set(help_catalogue.colnames)
in_both = in_help.union(in_brown)
only_in_brown = in_brown - in_help

for col in brown_catalogue.colnames:
    brown_catalogue[col].name = "brown_{}".format(col)
    
for col in help_catalogue.colnames:
    help_catalogue[col].name = "help_{}".format(col)

## 1.1 join the two catalogues taking brown if both are available

In [ ]:
master_catalogue = join(cross_id, brown_catalogue, join_type='left')
master_catalogue = join(master_catalogue, help_catalogue, join_type='left')
             



In [ ]:
for col in in_both:
    has_brown = (master_catalogue['brown_help_id'] != '-1')
    master_catalogue["help_{}".format(col)][has_brown] = master_catalogue["brown_{}".format(col)][has_brown]
    master_catalogue["help_{}".format(col)].name = col
    
for col in only_in_brown:
    master_catalogue["brown_{}".format(col)].name = col

###  - crop to HELP area

In [ ]:
help_bootes_moc = MOC(filename="../../dmu2_dmu2_field_coverages/Bootes_MOC.fits")
in_HELP_mask = inMoc(
    master_catalogue['ra'], master_catalogue['brown_dec'],
    help_bootes_moc 
)

master_catalogue = master_catalogue[in_HELP_mask]

In [ ]:
master_catalogue = master_catalogue.filled()
master_catalogue.write("{}/master_catalogue_bootes{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)